## Make Potential Publication Plots for WDS-Gaia Paper

#### Read in the wds table, query results table and the sorted results (so we can look at all the objects together as well as the sorted systems)

### Plot 1) Projection of the WDS Sources on the Sky 

### Plot 2) Projection of the WDS-Gaia Sources on the Sky

a] Both together (distinguished by color)

b] Just the physical pairs

c] Just the non-physical pairs

### Plot 3) Angular Separation Histogram
a] Both together (distinguished by color)

b] Just the physical pairs

c] Just the non-physical pairs

### Plot 4) Physical Separation Histogram

a] Both together (distinguished by color)

b] Just the physical pairs

c] Just the non-physical pairs

### Plot 5) Delta-Mag Histogram

a] Both together (distinguished by color)

b] Just the physical pairs

c] Just the non-physical pairs

### Plot 6) Mag vs Mag plot

a] Both together (distinguished by color)

b] Just the physical pairs

c] Just the non-physical pairs

### Plot 7) Color Magnitude Diagram

a] Both together (distinguished by color)

b] Just the physical pairs

c] Just the non-physical pairs